In [2]:
import glob
import os
import imageio
import skimage as ski
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from skimage.feature import SIFT, match_descriptors
from sklearn.cluster import KMeans
import pickle
"""
Resources used:
    https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.SIFT
"""

'\nResources used:\n    https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.SIFT\n'

In [3]:
def grey_list(paths):
    
    gray_list =[] #for path in paths:
    detector_list = []
    i = 0
    for path in paths:    
        image = imageio.v2.imread(path)
        gray_image = ski.color.rgb2gray(image) #this turns each pixel into a float 0-1
        gray_list.append(gray_image)
        sift_stuff(gray_image, detector_list)
    return detector_list

def sift_stuff(image, list):
    #descriptors are the local image gradient info, around the keypoint
    descriptor_extractor= SIFT()
    descriptor_extractor.detect_and_extract(image)
    list.append(descriptor_extractor.descriptors) # appending x amount of descriptors, each descriptor has 128 features

In [4]:

paths = sorted(glob.glob('Childrens-Books/*.jpg'))

sift_list = grey_list(paths)

In [5]:
with open('data.pkl', 'wb') as f:
    pickle.dump(sift_list, f)

In [6]:
for i, item in enumerate(sift_list):
    if item is not None and len(item) > 0:
        continue
    else:
        print("value not valid")

In [7]:

    #https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
kmeans = KMeans(n_clusters=1000)
flat_array = np.vstack(sift_list)
mean_list = kmeans.fit(flat_array) #needs a 1D array

In [9]:
with open('kmeans_data.pkl', 'wb') as f:
    pickle.dump(kmeans, f)

In [11]:

    
cluster_list = []
for item in sift_list:
    if item is not None and len(item) > 0:
        val = kmeans.predict(item)
        cluster_list.append(val)
    else: 
        print(item, "not valid")

In [19]:
print(flat_array.shape)
print(len(sift_list[0]))
print((cluster_list[0])) # returns for image 1, which cluster labels it aligns with

(387790, 128)
306
573


In [27]:
str_list = []
for index, cluster in enumerate(cluster_list):
    local_string = ""
    for x in range(len(cluster)):
        local_string += str(cluster[x]) + " "
    str_list.append(local_string)
    
    

In [28]:
str_list[0]

'727 0 274 395 345 60 354 723 795 462 795 159 795 723 795 696 829 805 185 829 220 345 60 829 829 467 60 345 219 865 656 579 579 52 795 795 898 789 837 687 903 73 73 812 63 205 33 173 242 500 65 287 736 786 716 100 33 558 33 279 500 888 812 686 960 833 418 100 386 68 463 606 448 288 940 543 18 107 452 753 846 583 73 715 960 753 902 100 732 223 721 828 828 338 659 159 280 418 990 556 300 159 781 367 930 871 612 781 3 534 552 870 534 116 816 534 870 84 116 473 695 39 258 159 159 159 39 159 50 913 713 338 976 605 84 781 258 534 473 534 617 258 428 159 913 973 124 124 472 963 67 460 429 138 33 429 426 100 975 715 281 744 960 732 279 967 279 279 100 967 960 960 731 279 279 279 426 426 416 190 619 624 515 960 342 134 960 963 963 889 889 744 443 426 364 268 426 850 911 911 145 697 145 529 903 963 472 812 472 812 963 152 436 10 746 857 152 198 436 436 690 122 27 494 303 783 368 4 599 65 736 732 732 435 816 473 229 732 358 358 847 39 591 728 870 258 159 258 393 795 258 173 534 998 534 505 534 99

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [30]:
vectorizer = TfidfVectorizer()